# API Main Functions and Use cases (Py to MindSphere)

Run `Import Libraries` and `Main set of Functions` to test the use cases.

## Import Libraries

In [6]:
import requests
import json
import base64

## Main set of Functions

In [7]:
#=========================================================
#--- Generate bearer_token
#=========================================================
def bearerToken():
    #--- App details from developer cockpit
    CLIENT_ID = "iiotuxpc-testdm-v1.0.0"
    CLIENT_SECRET = "UvwnNgdc8Y0KsE7IZSFy2Jp28kQGzAd0fEHXcDOWbIX"
    APP_NAME = "testdm"
    APP_VERSION = "v1.0.0"
    HOST_TENANT = "iiotuxpc"
    USER_TENANT = "iiotuxpc"
    credential_create_time = "23/02/2023::18:53"

    #--- generate access token
    id = CLIENT_ID
    secret = CLIENT_SECRET
    encode_msg = id + ":" + secret
    token = base64.b64encode(encode_msg.encode("utf-8")).decode("utf-8")
    url = "https://gateway.eu1.mindsphere.io/api/technicaltokenmanager/v3/oauth/token"

    payload = json.dumps({
    "grant_type": "client_credentials",
    "appName": APP_NAME,
    "appVersion": APP_VERSION,
    "hostTenant": HOST_TENANT,
    "userTenant": USER_TENANT
    })
    headers = {
    'X-SPACE-AUTH-KEY': "Basic " + token,
    'Content-Type': 'application/json'
    }

    token_response = requests.request("POST", url, headers=headers, data=payload)
    # print(token_response.text)
    bearer_token = json.loads(token_response.text)["access_token"]
    return(bearer_token)
#------------------------------------------------------------------------------------------------------

#=========================================================
#--- Find AssetID using Asset name with GET API
#=========================================================

def get_assetid(bearer_token,asset_name):
    #--- get list of assets
    bearer_token = bearerToken()
    url = "https://gateway.eu1.mindsphere.io/api/assetmanagement/v3/assets"
    payload={}
    headers = {
    'Authorization': 'Bearer ' + bearer_token
    }
    assets_list_response = requests.request("GET", url, headers=headers, data=payload)  # GET API call for assets_list
    assets_list=json.loads(assets_list_response.text)
    assets_count = assets_list["page"]["totalElements"]     # total number of assets available
    
    #--- find the asset_name and the corresponding ID in the assets_list
    for ii in range(0,assets_count):
        if assets_list["_embedded"]["assets"][ii]["name"] == asset_name:
            asset_Id = assets_list["_embedded"]["assets"][ii]["assetId"]
            # print(asset_name,asset_Id)
            break
    else: print("Error:\n\tAsset not found in the assets list.\n\tVerify the accuracy of asset name and the assets list. Asset name is case sensitive.")
    return(asset_Id)
#------------------------------------------------------------------------------------------------------

#=========================================================
#--- GET time series data
#=========================================================

# get time series data from MindSphere specific to the aspect of a asset and query condition
def get_timeseries(asset_name, aspect_name, from_time, to_time):
    bearer_token = bearerToken()    # expires in 1799 seconds
    asset_Id = get_assetid(bearer_token, asset_name)
    data_condition = "?from=" + from_time + "&to=" + to_time    # query condition with from and to time period
    end_point = "iottimeseries/v3/timeseries/" + asset_Id + "/" + aspect_name + data_condition
    base_url = "https://gateway.eu1.mindsphere.io/api/"
    url = base_url + end_point

    payload={}
    headers = {
     'Authorization': 'Bearer ' + bearer_token
    }

    time_series_response = requests.request("GET", url, headers=headers, data=payload)  # GET API call
    print(time_series_response.text)
    time_series=json.loads(time_series_response.text)
    return(time_series)

#------------------------------------------------------------------------------------------------------

#=========================================================
#--- PUT time series data
#=========================================================

# put time series data into MindSphere specific to the aspect of a asset
def put_timeseries(asset_name, aspect_name, payload):
    bearer_token = bearerToken()    # expires in 1799 seconds
    asset_Id = get_assetid(bearer_token, asset_name)
    end_point = "iottimeseries/v3/timeseries/" + asset_Id + "/" + aspect_name
    base_url = "https://gateway.eu1.mindsphere.io/api/"
    url = base_url + end_point

    headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': "Bearer " + bearer_token
    }

    response = requests.request("PUT", url, headers=headers, data=payload)  # PUT API call
    if not response.text:
        print("Data PUT successfully.")
    else:print("Error:\n\t",response.text)

#------------------------------------------------------------------------------------------------------


## Use Cases

### Generate bearer token

In [8]:
bearer_token = bearerToken()
print(bearer_token)

eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vaWlvdHV4cGMucGlhbS5ldTEubWluZHNwaGVyZS5pby90b2tlbl9rZXlzIiwia2lkIjoia2V5LWlkLTMiLCJ0eXAiOiJKV1QifQ.eyJqdGkiOiJjMDFhODRhNmUwZjI0NzBlYmVmNTI4NmI3MTdmMDcwZSIsInN1YiI6Imlpb3R1eHBjLXRlc3RkbS12MS4wLjAzOSIsInNjb3BlIjpbImlhbS1hY3Rpb24uY2xpZW50X2NyZWRlbnRpYWxzLnVzZXItaW1wZXJzb25hdGlvbiIsIm1kc3A6Y29yZTpBZG1pbjNyZFBhcnR5VGVjaFVzZXIiXSwiY2xpZW50X2lkIjoiaWlvdHV4cGMtdGVzdGRtLXYxLjAuMDM5IiwiY2lkIjoiaWlvdHV4cGMtdGVzdGRtLXYxLjAuMDM5IiwiYXpwIjoiaWlvdHV4cGMtdGVzdGRtLXYxLjAuMDM5IiwiZ3JhbnRfdHlwZSI6ImNsaWVudF9jcmVkZW50aWFscyIsInJldl9zaWciOiI3ODA5MTVhNCIsImlhdCI6MTY3NzQzMTE1OSwiZXhwIjoxNjc3NDMyOTU5LCJpc3MiOiJodHRwczovL2lpb3R1eHBjLnBpYW0uZXUxLm1pbmRzcGhlcmUuaW8vb2F1dGgvdG9rZW4iLCJ6aWQiOiJpaW90dXhwYyIsImF1ZCI6WyJpYW0tYWN0aW9uLmNsaWVudF9jcmVkZW50aWFscyIsImlpb3R1eHBjLXRlc3RkbS12MS4wLjAzOSJdLCJ0ZW4iOiJpaW90dXhwYyIsInNjaGVtYXMiOlsidXJuOnNpZW1lbnM6bWluZHNwaGVyZTppYW06djEiXSwiY2F0IjoiY2xpZW50LXRva2VuOnYxIn0.KLoog5hLTb9rGXe2s6sGG69v4ml00_evC1yVwEBG2bFvB6MIblOxuC4BDlmr22PfHNqPr

### Find Asset ID

In [108]:
bearer_token = bearerToken()
asset_Id = get_assetid(bearer_token,"machine")
# print(asset_Id)

### GET timeseries data

In [ ]:
# get list of aspects a asset
asset_name = "machine"
aspect_name = "real_log"
from_time = "2023-02-20T02:00:01.001Z"
to_time = "2023-02-20T02:02:01.001Z"

time_series = get_timeseries(asset_name, aspect_name, from_time, to_time)
# print(time_series)

### PUT timeseries data

In [114]:
#--- create a table from existing txt file or log file


#--- process the table to desired input format


#--- convert the table into json dictionary
payload = json.dumps([
    {
    "_time":"2023-02-20T02:01:01.001Z",
    "activity_type":"started",
    "machine_id":"machine_1",
    "part_id":"100",
    "queue":"queue 1",
    "timestamp":"2023-02-20T02:01:01.001Z"
    },
    {
    "_time":"2023-02-20T02:02:01.001Z",
    "activity_type":"finished",
    "machine_id":"machine_1",
    "part_id":"100",
    "queue":"queue 1",
    "timestamp":"2023-02-20T02:02:01.001Z"
    }
])

#--- call PUT API
asset_name = "machine"
aspect_name = "real_log"
put_timeseries(asset_name, aspect_name, payload)


Data PUT successfully.


# Testing Modules

### Initial steps
- create an app in the developer cockpit
- use `siemens.com` as direct url
- use /** as endpoint
- setup MindSPhere API roles for the user in the App roles section of Authorisation Management tab
- save and register
- goto App credentials in Authorization management tab and click issue access to get Client ID and Client secret

When application hosting type is `SELF_HOSTED`, the variables must be configured on server.

## App credentials and bearer_token for testdm (internal name)

In [96]:
#--- App details
CLIENT_ID = "iiotuxpc-testdm-v1.0.0"
CLIENT_SECRET = "UvwnNgdc8Y0KsE7IZSFy2Jp28kQGzAd0fEHXcDOWbIX"
APP_NAME = "testdm"
APP_VERSION = "v1.0.0"
HOST_TENANT = "iiotuxpc"
USER_TENANT = "iiotuxpc"
credential_create_time = "23/02/2023::18:53"

#--- generate access token
import requests
import json
import base64

id = CLIENT_ID
secret = CLIENT_SECRET
encode_msg = id + ":" + secret
token = base64.b64encode(encode_msg.encode("utf-8")).decode("utf-8")

url = "https://gateway.eu1.mindsphere.io/api/technicaltokenmanager/v3/oauth/token"

payload = json.dumps({
  "grant_type": "client_credentials",
  "appName": APP_NAME,
  "appVersion": APP_VERSION,
  "hostTenant": HOST_TENANT,
  "userTenant": USER_TENANT
})
headers = {
  'X-SPACE-AUTH-KEY': "Basic " + token,
  'Content-Type': 'application/json'
}

token_response = requests.request("POST", url, headers=headers, data=payload)

print(token_response.text)
bearer_token = json.loads(token_response.text)["access_token"]
print(bearer_token)

{"access_token":"eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vaWlvdHV4cGMucGlhbS5ldTEubWluZHNwaGVyZS5pby90b2tlbl9rZXlzIiwia2lkIjoia2V5LWlkLTMiLCJ0eXAiOiJKV1QifQ.eyJqdGkiOiIwZjU2MjdlZmMwOWE0MGRiYTdiMTc0MWM3ZmYwZGJiOSIsInN1YiI6Imlpb3R1eHBjLXRlc3RkbS12MS4wLjAzOSIsInNjb3BlIjpbImlhbS1hY3Rpb24uY2xpZW50X2NyZWRlbnRpYWxzLnVzZXItaW1wZXJzb25hdGlvbiIsIm1kc3A6Y29yZTpBZG1pbjNyZFBhcnR5VGVjaFVzZXIiXSwiY2xpZW50X2lkIjoiaWlvdHV4cGMtdGVzdGRtLXYxLjAuMDM5IiwiY2lkIjoiaWlvdHV4cGMtdGVzdGRtLXYxLjAuMDM5IiwiYXpwIjoiaWlvdHV4cGMtdGVzdGRtLXYxLjAuMDM5IiwiZ3JhbnRfdHlwZSI6ImNsaWVudF9jcmVkZW50aWFscyIsInJldl9zaWciOiI3ODA5MTVhNCIsImlhdCI6MTY3NzM3NDcwNSwiZXhwIjoxNjc3Mzc2NTA1LCJpc3MiOiJodHRwczovL2lpb3R1eHBjLnBpYW0uZXUxLm1pbmRzcGhlcmUuaW8vb2F1dGgvdG9rZW4iLCJ6aWQiOiJpaW90dXhwYyIsImF1ZCI6WyJpYW0tYWN0aW9uLmNsaWVudF9jcmVkZW50aWFscyIsImlpb3R1eHBjLXRlc3RkbS12MS4wLjAzOSJdLCJ0ZW4iOiJpaW90dXhwYyIsInNjaGVtYXMiOlsidXJuOnNpZW1lbnM6bWluZHNwaGVyZTppYW06djEiXSwiY2F0IjoiY2xpZW50LXRva2VuOnYxIn0.o6GbAhXLtzJlSATtMZjuzsJfZCHtEtHyy8a3Fh4gRtr1HhhYHglZ

## get assets list

In [8]:
# get list of assets
url = "https://gateway.eu1.mindsphere.io/api/assetmanagement/v3/assets"

payload={}
headers = {
  'Authorization': 'Bearer ' + bearer_token
}

assets_list_response = requests.request("GET", url, headers=headers, data=payload)
assets_list=json.loads(assets_list_response.text)


In [62]:
#========================================
#--- accessing assets list data
#========================================

#-- getting total number of assets in the list
assets_count = assets_list["page"]["totalElements"]
# print(assets_count)

# #--- getting name of a specific asset in the list
# asset_name = assets_list["_embedded"]["assets"][2]["name"]
# print(asset_name)

# #--- getting id of a specific asset in the list
# asset_Id = assets_list["_embedded"]["assets"][2]["assetId"]
# print(asset_Id)

#--- finding asset id using name from the assets list
asset_name = "machine"
for ii in range(0,assets_count):
    if assets_list["_embedded"]["assets"][ii]["name"] == asset_name:
        asset_Id = assets_list["_embedded"]["assets"][ii]["assetId"]
        print(asset_name,asset_Id)
        break
else: print("Error:\n\tAsset not found in the assets list.\n\tVerify the accuracy of asset name and the assets list. Asset name is case sensitive.")


machine 60bf86d1d051416f853b1e22f3714b8c


## get Aspects list of a asset

In [46]:
# get list of aspects of a asset
init_end_point = "assetmanagement/v3/assets/"
data_needed = "/aspects/"
end_point = init_end_point + asset_Id + data_needed

base_url = "https://gateway.eu1.mindsphere.io/api/"
url = base_url + end_point

payload={}
headers = {
  'Authorization': 'Bearer ' + bearer_token
}

aspects_list_response = requests.request("GET", url, headers=headers, data=payload)
# print(aspects_list_response.text)
aspects_list=json.loads(aspects_list_response.text)

In [57]:
#========================================
#--- accessing aspects list data
#========================================

#-- getting total number of aspects in the list
aspects_count = aspects_list["page"]["totalElements"]
# print(aspects_count)

# #--- getting name of a specific asset in the list
# aspect_name = aspects_list["_embedded"]["aspects"][1]["name"]
# print(aspect_name)

# #--- getting id of a specific asset in the list
# aspect_Id = aspects_list["_embedded"]["aspects"][1]["aspectId"]
# print(aspect_Id)

#--- finding aspect id using name from the assests list
aspect_name = "status"
for ii in range(0,aspects_count):
    if aspects_list["_embedded"]["aspects"][ii]["name"] == aspect_name:
        aspect_Id = aspects_list["_embedded"]["aspects"][ii]["aspectId"]
        # print(aspect_name,aspect_Id)
        break
else: print("Error:\n\tAspect not found in the assets list.\n\tVerify the accuracy of the asset ID, aspect name and the aspects list. Aspect name is case sensitive.")

## get Variables list of a asset (not defined as a part of a aspect)

In [18]:
# get list of asset variables defined outside aspects
init_point = "assetmanagement/v3/assets/"
data_needed = "/variables"
end_point = asset_end_point + asset_Id + data_needed

base_url = "https://gateway.eu1.mindsphere.io/api/"
url = base_url + end_point

payload={}
headers = {
  'Authorization': 'Bearer ' + bearer_token
}

variables_list_response = requests.request("GET", url, headers=headers, data=payload)
# print(variables_list_response.text)
variables_list=json.loads(variables_list_response.text)

NameError: name 'asset_end_point' is not defined

In [ ]:
#========================================
#--- accessing variables list datadefined outside aspects
#========================================

#-- getting total number of variables in the list
variables_count = variables_list["page"]["totalElements"]
print(variables_count)

#--- getting name of a specific asset in the list
variable_name = variables_list["_embedded"]["variables"][0]["name"]
print(variable_name)

#--- getting id of a specific asset in the list
variable_Id = variables_list["_embedded"]["variables"][0]["variableId"] # gives out error if no variables declared
print(variable_Id)

## get iot timeseries data (ISO8601 format: YYYY-MM-DDTHH:MM:SS.SSSZ)

In [81]:
# get list of aspects a asset
asset_name = "machine"
asset_Id = "60bf86d1d051416f853b1e22f3714b8c"
aspect_name = "real_log"
from_time = "2023-02-20T02:00:01.001Z"
to_time = "2023-02-20T02:02:01.001Z"
data_condition = "?from=" + from_time + "&to=" + to_time
end_point = "iottimeseries/v3/timeseries/" + asset_Id + "/" + aspect_name + data_condition

base_url = "https://gateway.eu1.mindsphere.io/api/"
url = base_url + end_point

payload={}
headers = {
  'Authorization': 'Bearer ' + bearer_token
}

time_series_response = requests.request("GET", url, headers=headers, data=payload)
print(time_series_response.text)
time_series=json.loads(time_series_response.text)
# print(time_series)

[{"activity_type":"started","machine_id":"machine_1","part_id":"100","_time":"2023-02-20T02:01:01.001Z","queue":"queue 1","timestamp":"2023-02-20T02:01:01.001Z"},{"activity_type":"finished","machine_id":"machine_1","part_id":"100","_time":"2023-02-20T02:02:01.001Z","queue":"queue 1","timestamp":"2023-02-20T02:02:01.001Z"}]


## put iot timeseries data (ISO8601 format: YYYY-MM-DDTHH:MM:SS.SSSZ)

- machine:real_log
- _time:YYYY-MM-DDTHH:MM:SS.SSSZ, activity_type:STRING, machine_id:STRING, part_id:STRING, queue:STRING, timestamp:YYYY-MM-DDTHH:MM:SS.SSSZ
- _time (MindSphere internal variable) is compulsary as we are writing to a time series database

In [91]:
#--- create a table from existing txt file or log file


#--- process the table to desired input format


#--- convert the table into json dictionary
payload = json.dumps([
    {
    "_time":"2023-02-20T02:01:01.001Z",
    "activity_type":"started",
    "machine_id":"machine_1",
    "part_id":"100",
    "queue":"queue 1",
    "timestamp":"2023-02-20T02:01:01.001Z"
    },
    {
    "_time":"2023-02-20T02:02:01.001Z",
    "activity_type":"finished",
    "machine_id":"machine_1",
    "part_id":"100",
    "queue":"queue 1",
    "timestamp":"2023-02-20T02:02:01.001Z"
    }
])

#--- process the API PUT sequence
asset_Id = "60bf86d1d051416f853b1e22f3714b8c"
aspect_name = "real_log"
end_point = "iottimeseries/v3/timeseries/" + asset_Id + "/" + aspect_name
base_url = "https://gateway.eu1.mindsphere.io/api/"

url = base_url + end_point
data_json = json.dumps(data)
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'Authorization': "Bearer " + bearer_token
}

response = requests.request("PUT", url, headers=headers, data=payload)
if not response.text:
  print("Data PUT successfully.")
else:print("Error:\n\t",response.text)

Data PUT successfully.
